# Scrape Simpsons episodes
This notebook scrapes all the episodes from The Simpsons.

## Libraries

In [ ]:
# Import all libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

## Scrape all episode tables

Get the text from all `wikitable plainrowheaders wikiepisodetable` tables.

In [ ]:
# Get request
url = 'https://en.wikipedia.org/wiki/List_of_The_Simpsons_episodes_(seasons_1%E2%80%9320)'
res = requests.get(url)
soup = BeautifulSoup(res.text, 'html.parser')

# Find all episode tables
tables = soup.find_all(
    name='table',
    attrs={
        'class': 'wikitable plainrowheaders wikiepisodetable'
    }
)

Compile all the tables into a single dataframe.

In [ ]:
# Columns to retrieve
cols = [
    'Title',
    'Original air date',
    'U.S. viewers (millions)'
]

# Read all tables and concat them together
for i, j in enumerate(tables):
        
    # Read HTML (omit movies)
    try:
        d = d[cols]
    except:
        pass
    
    # Init dataframe
    if i == 0:

        # Read HTML
        df = d
    
    # Append subsequent frames
    else:

        # Concat to `df`
        df = pd.concat([df, d])

Clean episodes dataframe

In [ ]:
# Rename columns
df.columns = [
    'title',
    'release',
    'viewers'
]

# Clean title column
df['title'] = df['title'].str.replace('"', '')

# Turn date to datetime
df['release'] = pd.to_datetime(
    arg=df['release'],
    format='%B %d, %Y'
)

# Clean viewers
df['viewers'] = df['viewers'].str.split('[').str[0].astype(float)

## Scrape Special Guests list